In [108]:
import numpy as np
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [109]:
X, y = make_classification(n_samples=1000, n_features=64, n_informative=5, n_redundant=2, n_classes=2, random_state=42)

# Chuyển đổi sang DataFrame để dễ nhìn
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df['target'] = y


x_train, x_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

x_train = x_train.values
y_train = y_train.values
x_test = x_test.values
y_test = y_test.values

In [110]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
train_loader = DataLoader(TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train)), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test)), batch_size=32, shuffle=False)

(800, 64) (200, 64) (800,) (200,)


In [111]:
class NeuralNetwork(nn.Module):
  def __init__(self, input_size = 64, hidden_size1 = 25, hidden_size2 = 15, output_size = 1):
    super(NeuralNetwork, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size1)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size1, hidden_size2)
    self.fc3 = nn.Linear(hidden_size2, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.fc3(out)
    out = self.sigmoid(out)
    return out

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=64, out_features=25, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=25, out_features=15, bias=True)
  (fc3): Linear(in_features=15, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [112]:
model = NeuralNetwork()
epochs = 10
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()
for epoch in range(epochs): # Added epoch variable for better logging
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.float()) # Convert data to float
        loss = criterion(output, target.float().unsqueeze(1)) # Convert target to float and add a dimension
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, target in test_loader:
            output = model(data.float()) # Convert data to float
            predicted = (output > 0.5).float() # Get binary predictions from probabilities
            correct += (predicted == target.float().unsqueeze(1)).sum().item() # Compare with target and add to correct count
            total += target.size(0)

        print(f"Epoch {epoch+1} Accuracy: {correct/total}") # Added epoch number to accuracy print

Epoch 1 Accuracy: 0.61
Epoch 2 Accuracy: 0.69
Epoch 3 Accuracy: 0.7
Epoch 4 Accuracy: 0.78
Epoch 5 Accuracy: 0.79
Epoch 6 Accuracy: 0.805
Epoch 7 Accuracy: 0.805
Epoch 8 Accuracy: 0.825
Epoch 9 Accuracy: 0.82
Epoch 10 Accuracy: 0.825


In [113]:
model = NeuralNetwork()
epochs = 10
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()
for epoch in range(epochs): # Added epoch variable for better logging
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.float()) # Convert data to float
        loss = criterion(output, target.float().unsqueeze(1)) # Convert target to float and add a dimension
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, target in test_loader:
            output = model(data.float()) # Convert data to float
            predicted = (output >= 0.5).float() # Get binary predictions from probabilities
            correct += (predicted == target.float().unsqueeze(1)).sum().item() # Compare with target and add to correct count
            total += target.size(0)

        print(f"Epoch {epoch+1} Accuracy: {correct/total}") # Added epoch number to accuracy print

Epoch 1 Accuracy: 0.66
Epoch 2 Accuracy: 0.69
Epoch 3 Accuracy: 0.765
Epoch 4 Accuracy: 0.785
Epoch 5 Accuracy: 0.805
Epoch 6 Accuracy: 0.815
Epoch 7 Accuracy: 0.83
Epoch 8 Accuracy: 0.825
Epoch 9 Accuracy: 0.825
Epoch 10 Accuracy: 0.83
